In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf

# 读取用户、电影数据

In [2]:
movies_df=pd.read_csv('ml-latest-small/movies.csv')

In [3]:
movies_df.tail()

,movieId,title,genres
9120,162672,Mohenjo Daro (2016),Adventure|Drama|Romance
9121,163056,Shin Godzilla (2016),Action|Adventure|Fantasy|Sci-Fi
9122,163949,The Beatles: Eight Days a Week - The Touring Y...,Documentary
9123,164977,The Gay Desperado (1936),Comedy
9124,164979,"Women of '69, Unboxed",Documentary


In [4]:
ratings_df=pd.read_csv('ml-latest-small/ratings.csv')

In [5]:
ratings_df.tail()

,userId,movieId,rating,timestamp
99999,671,6268,2.5,1065579370
100000,671,6269,4.0,1065149201
100001,671,6365,4.0,1070940363
100002,671,6385,2.5,1070979663
100003,671,6565,3.5,1074784724


# 获得movie-user-rating表    

In [6]:
movies_df['movieIndex']=movies_df.index

In [7]:
movies_df.tail()

,movieId,title,genres,movieIndex
9120,162672,Mohenjo Daro (2016),Adventure|Drama|Romance,9120
9121,163056,Shin Godzilla (2016),Action|Adventure|Fantasy|Sci-Fi,9121
9122,163949,The Beatles: Eight Days a Week - The Touring Y...,Documentary,9122
9123,164977,The Gay Desperado (1936),Comedy,9123
9124,164979,"Women of '69, Unboxed",Documentary,9124


In [8]:
movies_df=movies_df[['movieIndex','movieId','title']]

In [9]:
movies_df.tail()

,movieIndex,movieId,title
9120,9120,162672,Mohenjo Daro (2016)
9121,9121,163056,Shin Godzilla (2016)
9122,9122,163949,The Beatles: Eight Days a Week - The Touring Y...
9123,9123,164977,The Gay Desperado (1936)
9124,9124,164979,"Women of '69, Unboxed"


In [10]:
ratings_df.tail()

,userId,movieId,rating,timestamp
99999,671,6268,2.5,1065579370
100000,671,6269,4.0,1065149201
100001,671,6365,4.0,1070940363
100002,671,6385,2.5,1070979663
100003,671,6565,3.5,1074784724


In [11]:
ratings_df=pd.merge(movies_df,ratings_df,on='movieId')

In [12]:
ratings_df.head()

,movieIndex,movieId,title,userId,rating,timestamp
0,0,1,Toy Story (1995),7,3.0,851866703
1,0,1,Toy Story (1995),9,4.0,938629179
2,0,1,Toy Story (1995),13,5.0,1331380058
3,0,1,Toy Story (1995),15,2.0,997938310
4,0,1,Toy Story (1995),19,3.0,855190091


In [13]:
movieNo=ratings_df['movieIndex'].max()+1

In [14]:
movieNo

9123

In [15]:
userNo=ratings_df['userId'].max()+1

In [16]:
userNo

672

In [17]:
rating=np.zeros([movieNo,userNo])

In [85]:
idx=0
total_length=ratings_df.shape[0]

for index,row in ratings_df.iterrows():
    rating[int(row['movieIndex']),int(row['userId'])]=int(row['rating'])
    idx+=1
    if idx%5000==0:
        print('processed:%d,left:%d' % (idx,total_length-idx))

processed:5000,left:95004
processed:10000,left:90004
processed:15000,left:85004
processed:20000,left:80004
processed:25000,left:75004
processed:30000,left:70004
processed:35000,left:65004
processed:40000,left:60004
processed:45000,left:55004
processed:50000,left:50004
processed:55000,left:45004
processed:60000,left:40004
processed:65000,left:35004
processed:70000,left:30004
processed:75000,left:25004
processed:80000,left:20004
processed:85000,left:15004
processed:90000,left:10004
processed:95000,left:5004
processed:100000,left:4


In [19]:
rating

array([[ 0.,  0.,  0., ...,  0.,  4.,  5.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       ..., 
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.]])

In [20]:
record=rating>0

In [21]:
record=np.array(record,dtype=int)

In [22]:
record

array([[0, 0, 0, ..., 0, 1, 1],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ..., 
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

# 对评分标准化

In [27]:
def normalizeRating(rating,record):
    m,n=rating.shape
    rating_mean=np.zeros((m,1))
    rating_norm=np.zeros((m,n))
    for i in range(m):
        idx=record[i,:]!=0
        rating_mean[i]=np.mean(rating[i,idx])
        rating_norm[i,:]=rating[i,:]-rating_mean[i]
    return rating_mean,rating_norm

In [28]:
rating_mean,rating_norm=normalizeRating(rating,record)

D:\Program Files\Anaconda3\envs\tensorflow\lib\site-packages\numpy\core\fromnumeric.py:2909: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
D:\Program Files\Anaconda3\envs\tensorflow\lib\site-packages\numpy\core\_methods.py:80: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [30]:
rating_mean=np.nan_to_num(rating_mean)

In [31]:
rating_norm=np.nan_to_num(rating_norm)

# 训练模型

In [32]:
num_features=10
X_variable=tf.Variable(tf.random_normal([movieNo,num_features],stddev=0.35))
Theta_variable=tf.Variable(tf.random_normal([userNo,num_features],stddev=0.35))

In [33]:
loss=1/2*tf.reduce_sum(((tf.matmul(X_variable,Theta_variable,transpose_b=True)-rating_norm)*record)**2)
+1/2*tf.reduce_sum(X_variable**2)+1/2*tf.reduce_sum(Theta_variable**2)

<tf.Tensor 'add:0' shape=() dtype=float32>

In [34]:
optimizer=tf.train.AdamOptimizer(1e-4)
train=optimizer.minimize(loss)

In [37]:
tf.summary.scalar('loss',loss)
summaryMerged=tf.summary.merge_all()
filename='./tensorboard_loss'
writer=tf.summary.FileWriter(filename)

In [69]:
sess=tf.Session()
init=tf.global_variables_initializer()
sess.run(init)

In [70]:
for i in range(10000):
    _,movie_summary=sess.run([train,summaryMerged])
    writer.add_summary(movie_summary,i)

In [71]:
X_result,Theta_result=sess.run([X_variable,Theta_variable])

In [72]:
predicts=np.dot(X_result,Theta_result.T)+rating_mean

In [73]:
error_total=np.sqrt(np.sum(((predicts-rating)*record)**2))

In [74]:
error_total

175.97139439157209

In [75]:
predicts.shape

(9123, 672)

In [83]:
idx=1
index_show=500
for i in range(671):
    if rating[index_show,i]!=0:
        print('Real rating:%d; Predicting:%.2f' % (rating[index_show,i],predicts[index_show,i]))
        idx+=1
    if(idx>20):break


Real rating:3; Predicting:2.95
Real rating:4; Predicting:3.72
Real rating:1; Predicting:0.90
Real rating:2; Predicting:2.22
Real rating:1; Predicting:1.14
Real rating:3; Predicting:3.17
Real rating:2; Predicting:2.32
Real rating:4; Predicting:3.66
Real rating:4; Predicting:3.81


# 展示最终结果

In [77]:
user_id=input('请输入需要推荐的用户编号：')

请输入需要推荐的用户编号：123


In [78]:
sortedResult=predicts[:,int(user_id)].argsort()[::-1]

In [79]:
sortedResult

array([3095, 8340, 7529, ..., 7897, 7976, 1372], dtype=int64)

In [80]:
idx=0
print("推荐给用户的20部电影：")
for i in sortedResult:
    print('rating: %.2f; movie:%s '% (predicts[i,int(user_id)],movies_df.iloc[i]['title']))
    idx+=1
    if idx==20:break

推荐给用户的20部电影：
rating: 8.24; movie:Shane (1953) 
rating: 8.15; movie:Monsters University (2013) 
rating: 8.08; movie:Four Lions (2010) 
rating: 8.06; movie:O Lucky Man! (1973) 
rating: 7.94; movie:Rivers and Tides (2001) 
rating: 7.92; movie:Baraka (1992) 
rating: 7.87; movie:I Love You, Man (2009) 
rating: 7.80; movie:Millennium Actress (Sennen joyû) (2001) 
rating: 7.73; movie:Searchers, The (1956) 
rating: 7.72; movie:Excision (2012) 
rating: 7.70; movie:Jin Roh: The Wolf Brigade (Jin-Rô) (1998) 
rating: 7.69; movie:Babette's Feast (Babettes gæstebud) (1987) 
rating: 7.68; movie:Threads (1984) 
rating: 7.64; movie:Safe (1995) 
rating: 7.61; movie:Flags of Our Fathers (2006) 
rating: 7.58; movie:Ponyo (Gake no ue no Ponyo) (2008) 
rating: 7.43; movie:Brother 2 (Brat 2) (2000) 
rating: 7.37; movie:Kids in the Hall: Brain Candy (1996) 
rating: 7.37; movie:Babylon 5: In the Beginning (1998) 
rating: 7.31; movie:Bad and the Beautiful, The (1952) 
